
This assumes you've read the idt notebook first: idt/idt.ipynb
This assumes you've read the stacks notebook first: stacks/kernel_stacks.ipynb
This assumes you've read the red zone notebook firest: stacks/red_zone.ipynb
This assumes you've read the idt/idt_swap.ipynb notebook first.

 
Here we show how to use a symbiote to force the page fault handler to use an ist stack as opposed to the default kernel stack. It turns out, this is very simple. All you have to do is set the interrupt stack table (IST) bit in the IDT entry for the page fault interrupt.

The Interrupt Stack Table (IST) mechanism of x86_64 is useful because it allows for an unconditional stack switch when an interrupt occurs. This is useful for symbiotes, because in general, x86_64 hardware (as used by the Linux kernel) takes different conditional behavior for interrupts that are delivered in user mode vs kernel mode. For example, in some cases, an interrupt that occurs when the CPU was in user mode will force a stack switch to the default kernel stack, while an interrupt that occurs when the CPU was in kernel mode will not force a stack switch. This is the case with page faults.

In [1]:
# Import the common code module found at explore/common_code/common.py
import sys
sys.path.append("../common_code/")
import common as com


Here we will modify the page fault handler to run on the double fault's IST stack as opposed to the default stack.

In [2]:
# Take a look at the page fault handler idt entry
PG_FT_IDX="14"
com.run_cmd("idt_tool -h")

CompletedProcess(args=['idt_tool', '-h'], returncode=0, stdout='', stderr='./idt_tool:\noptions:\n\t-a <addr>: address of idt, current loaded assumed if not provided\n\tc:         copy idt return ptr to copy on kern pg\n\tg:         get current idtr\n\th:         print this help msg\n\ti:         install idt (swing idtr)\n\tm <ist_enable|ist_disable|addr:0xaddr>: modify idt entry\n\tp:         print\n\tv <dec#>:  vector number for print / modify\n\tz <df|tf>: which mitigation to copy to kern page\n\nexamples:\n\ttaskset -c 1 ./idt_tool -g\n\ttaskset -c 1 ./idt_tool -c\n\t./idt_tool -z tf\n\t./idt_tool -a ffffc90000986000 -m addr:0xffffc9000098d000 -v 14\n\ttaskset -c 1 ./idt_tool -a ffffc90000986000 -i\n\ndf mitigation workflow:\n\ttaskset -c 0 ./idt_tool -g\n\ttaskset -c 0 ./idt_tool -c\n\t./idt_tool -z df\n\t./idt_tool -a ffffc900002ef000 -m addr:0xffffc90000317000 -v 8\n\ttaskset -c 0 ./idt_tool -a ffffc900002ef000 -i\n')